In [1]:
%load_ext autoreload
%autoreload 2

In [175]:
import sys
sys.path.append('..')

import numpy as np
import torch

from src.grus import GRU_Cell

# TORCH CONFIGURATION
device = torch.device("cpu")

In [193]:
# Input
input_size = 4
output_size = 4
hidden_size = 3
batch_size = 2
current_state = torch.zeros(hidden_size, batch_size)

# attach to device
current_state.to(device);

In [206]:
# data preparation
from sklearn.preprocessing import OneHotEncoder
string = 'abcd'*20
c2n = {'a': 0, 'b': 1, 'c': 2, 'd': 3}
n2c = {v:k for k,v in c2n.items()}
string_numb = np.array([c2n[char] for char in string]).reshape(-1, 1)
encoder = OneHotEncoder()
input_data = encoder.fit_transform(string_numb).todense()

batches = 40
input_data = torch.tensor(input_data, dtype=torch.float32).reshape(batches, input_size, batch_size)

In [207]:
# utils
def decode_character(onehot):
    indx_max = torch.argmax(onehot.squeeze()).tolist()
    return n2c[indx_max]

# model 
gru = GRU_Cell(hidden_size, input_size, batch_size, output_size)

for batch_index in range(batches):
    batch = input_data[batch_index]
    assert batch.shape == (input_size, batch_size)
    activation, current_state = gru.forward(batch, current_state)